In [ ]:
import jax
import time
import jax.numpy as jnp
from flax import nnx
import optax
from typing import Tuple

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-08-25 01:31:18--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.03s   

2024-08-25 01:31:19 (40.5 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
# Hyperparameters
batch_size = 32
block_size = 128
max_iters = 500
eval_interval = 100
learning_rate = 3e-4
eval_iters = 50
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.2

In [ ]:
# Data loading and preprocessing
def load_data():
    with open('input.txt', 'r', encoding='utf-8') as f:
        text = f.read()
    chars = sorted(list(set(text)))
    vocab_size = len(chars)
    stoi = {ch: i for i, ch in enumerate(chars)}
    itos = {i: ch for i, ch in enumerate(chars)}
    encode = lambda s: [stoi[c] for c in s]
    decode = lambda l: ''.join([itos[i] for i in l])
    data = jnp.array(encode(text), dtype=jnp.int32)
    n = int(0.9 * len(data))
    train_data = data[:n]
    val_data = data[n:]
    return train_data, val_data, encode, decode, vocab_size

In [ ]:
train_data, val_data, encode, decode, vocab_size = load_data()

In [ ]:
def get_batch(data, key):
    data_size = data.shape[0]
    key, subkey = jax.random.split(key)
    ix = jax.random.randint(subkey, (batch_size,), 0, data_size - block_size)
    x = jnp.take(data, jnp.arange(block_size)[None, :] + ix[:, None], axis=0)
    y = jnp.take(data, jnp.arange(1, block_size + 1)[None, :] + ix[:, None], axis=0)
    return x, y, key

In [ ]:
class Head(nnx.Module):
    def __init__(self, head_size: int, rngs: nnx.Rngs):
        self.key = nnx.Linear(n_embd, head_size, use_bias=False, rngs=rngs)
        self.query = nnx.Linear(n_embd, head_size, use_bias=False, rngs=rngs)
        self.value = nnx.Linear(n_embd, head_size, use_bias=False, rngs=rngs)
        self.dropout = nnx.Dropout(dropout, rngs=rngs)

    def __call__(self, x, training: bool = False):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)

        att = (q @ k.transpose(0, 2, 1)) * (k.shape[-1] ** -0.5)
        att = jnp.where(jnp.tril(jnp.ones((T, T))) == 0, float('-inf'), att)
        att = jax.nn.softmax(att, axis=-1)
        att = self.dropout(att, deterministic=not training)

        return att @ v

class MultiHeadAttention(nnx.Module):
    def __init__(self, num_heads: int, head_size: int, rngs: nnx.Rngs):
        self.heads = [Head(head_size, rngs=rngs) for _ in range(num_heads)]
        self.proj = nnx.Linear(num_heads * head_size, n_embd, rngs=rngs)
        self.dropout = nnx.Dropout(dropout, rngs=rngs)

    def __call__(self, x, training: bool = False):
        out = jnp.concatenate([h(x, training=training) for h in self.heads], axis=-1)
        out = self.dropout(self.proj(out), deterministic=not training)
        return out

# class FeedForward(nnx.Module):
#     def __init__(self, rngs: nnx.Rngs):
#         self.net = nnx.Sequential([
#             nnx.Linear(n_embd, 4 * n_embd, rngs=rngs),
#             jax.nn.relu,
#             nnx.Linear(4 * n_embd, n_embd, rngs=rngs),
#             nnx.Dropout(dropout, rngs=rngs)
#         ])

#     def __call__(self, x):
#         return self.net(x)

class FeedForward(nnx.Module):
    def __init__(self, rngs: nnx.Rngs):
        self.net = [
            nnx.Linear(n_embd, 4 * n_embd, rngs=rngs),
            nnx.Dropout(dropout, rngs=rngs),
            nnx.Linear(4 * n_embd, n_embd, rngs=rngs),
            nnx.Dropout(dropout, rngs=rngs)
        ]

    def __call__(self, x, training: bool = False):
        for layer in self.net:
            if isinstance(layer, nnx.Dropout):
                x = layer(x, deterministic=not training)
            else:
                x = layer(x)
            if layer == self.net[0]:  # After the first linear layer
                x = jax.nn.relu(x)
        return x

class Block(nnx.Module):
    def __init__(self, rngs: nnx.Rngs):
        self.sa = MultiHeadAttention(n_head, n_embd // n_head, rngs=rngs)
        self.ffwd = FeedForward(rngs=rngs)
        self.ln1 = nnx.LayerNorm(n_embd, rngs=rngs)
        self.ln2 = nnx.LayerNorm(n_embd, rngs=rngs)

    def __call__(self, x, training: bool = False):
        x = x + self.sa(self.ln1(x), training=training)
        x = x + self.ffwd(self.ln2(x), training=training)
        return x

class GPTLanguageModel(nnx.Module):
    def __init__(self, vocab_size: int, rngs: nnx.Rngs):
        self.token_embedding_table = nnx.Embed(vocab_size, n_embd, rngs=rngs)
        self.position_embedding_table = nnx.Embed(block_size, n_embd, rngs=rngs)
        # self.blocks = nnx.Sequential([Block(rngs=rngs) for _ in range(n_layer)])
        self.blocks = [Block(rngs=rngs) for _ in range(n_layer)]
        self.ln_f = nnx.LayerNorm(n_embd, rngs=rngs)
        self.lm_head = nnx.Linear(n_embd, vocab_size, rngs=rngs)

    def __call__(self, idx, targets=None, training: bool = False):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(jnp.arange(T))
        x = tok_emb + pos_emb

        # x = self.blocks(x)
        for block in self.blocks:
            x = block(x, training=training)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.reshape(B*T, C)
            targets = targets.reshape(B*T)
            loss = optax.softmax_cross_entropy_with_integer_labels(logits, targets).mean()

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = jax.nn.softmax(logits, axis=-1)
            idx_next = jax.random.categorical(jax.random.PRNGKey(0), probs, axis=-1)
            idx = jnp.concatenate([idx, idx_next[:, None]], axis=1)
        return idx



In [ ]:
# Initialize model and optimizer
model = GPTLanguageModel(vocab_size, rngs=nnx.Rngs(0))
optimizer = nnx.Optimizer(model, optax.adam(learning_rate))

@nnx.jit
def train_step(model, optimizer, x, y):
    def loss_fn(model):
        _, loss = model(x, y, training=True)
        return loss

    loss, grads = nnx.value_and_grad(loss_fn)(model)
    optimizer.update(grads)

    return loss

@nnx.jit
def eval_model(model, x, y):
    _, loss = model(x, y, training=False)
    return loss

# Training loop
print("Starting training...")
key = jax.random.PRNGKey(0)
start_time = time.time()
for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = {'train': 0.0, 'val': 0.0}
        for split in ['train', 'val']:
            data = train_data if split == 'train' else val_data
            for _ in range(eval_iters):
                key, subkey = jax.random.split(key)
                x, y, key = get_batch(data, subkey)
                loss = eval_model(model, x, y)
                losses[split] += loss / eval_iters
        print(f"Step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}, time: {time.time() - start_time:.2f}s")

    key, subkey = jax.random.split(key)
    x, y = get_batch(train_data, subkey)
    loss = train_step(model, optimizer, x, y)

    if iter % 10 == 0:
        print(f"Iteration {iter}, Loss: {loss:.4f}", end='\r', flush=True)

print("\nTraining completed.")



Starting training...
Step 0: train loss 4.7810, val loss 4.7506, time: 19.93s


ValueError: too many values to unpack (expected 2)

In [ ]:
# Generate sample text
print("\nGenerating sample text...")
context = jnp.zeros((1, 1), dtype=jnp.int32)
generated = model.generate(context, max_new_tokens=100)
print(decode(generated[0].tolist()))

# Helper functions (make sure these are defined)
# def get_batch(data):
#     ix = jax.random.randint(jax.random.PRNGKey(0), (batch_size,), 0, len(data) - block_size)
#     x = jnp.stack([data[i:i+block_size] for i in ix])
#     y = jnp.stack([data[i+1:i+block_size+1] for i in ix])
#     return x, y

